In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
batch_size = 2
# load data
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
    
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
    
        return normalized_sample, label
        
train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_44178/773740783.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import torch
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        
        in_channels = 2
        in_features = 128
        
        # define convolutional layers
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        
        # efine the fully connected layers
        self.fc1 = nn.Linear(128 * (in_features // 4), 256)  # in_features // 4 due to max pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # input shape: (batch_size, 2, 128)
        
        # convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        # flatten (before fully connected layers)
        x = x.view(x.size(0), -1)
        
        # fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

# initialize model
num_classes = 2 #BPSK / QPSK
model = CNN1D(num_classes)

print(model) #model info

CNN1D(
  (conv1): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu2): ReLU()
  (maxpool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


learning_rate = 0.001
num_epochs = 10

# loss function and optimizer definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training
for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    
    for data, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    
    # print loss
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')

# evaluate the model on test data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # predicted and actual labels
        for i in range(len(labels)):
            print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_44178/773740783.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_44178/773740783.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_44178/773740783.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(ax

Epoch [1/10] Loss: 0.0009
Epoch [2/10] Loss: 0.0030
Epoch [3/10] Loss: 0.1766
Epoch [4/10] Loss: 0.0024
Epoch [5/10] Loss: 0.1111
Epoch [6/10] Loss: 0.0006
Epoch [7/10] Loss: 0.0000
Epoch [8/10] Loss: 0.0009
Epoch [9/10] Loss: 0.0000
Epoch [10/10] Loss: 0.0000
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actua

In [4]:
print(total)

400


In [5]:
torch.save(model, "trainedModel")